In [203]:
import requests
import csv
import time
from bs4 import BeautifulSoup
from collections import Counter
import praw
import pandas as pd
import datetime as dt #only if you want to analyze the date created feature
import json 
from types import SimpleNamespace
import sys
from nltk.tokenize import TreebankWordTokenizer
# !pip install textblob
from textblob import TextBlob
# !pip install tabulate
from tabulate import tabulate

Fetch the CSV files.
1. Mental, Anxiety and Unrelated are all posted by ANXIOUS users
2. Control posts are posted by CONTROL users


In [197]:
df_anx_1 = pd.read_csv("dataset/reddit-posts-mental.csv", index_col=0)
df_anx_2 = pd.read_csv("dataset/reddit-posts-anxiety.csv", index_col=0)
df_anx_3 = pd.read_csv("dataset/reddit-posts-unrelated.csv", index_col=0)
df_anxiety_group = pd.concat([df_anx_1, df_anx_2, df_anx_3])
df_control_group = pd.read_csv("dataset/reddit-control.csv", index_col=0)

Create the function to tokenize a string

In [166]:
def get_tokens(string, to_lower): # TODO : Change to BERT / explore more as we see fit
    ''' String ---> Tokenize ---> Lowercase (if set to true) '''
    try:
        return TreebankWordTokenizer().tokenize(string) if to_lower is False else \
            [token.lower() for token in TreebankWordTokenizer().tokenize(string)]
    except:
        return []

Tokenize the title and body and add it to the feature dataframes

In [198]:
df_control_group['f-title-tokens'] = [get_tokens(title, True) for title in df_control_group['title']]
print("Done")
df_control_group['f-body-tokens'] = [get_tokens(body, True) for body in df_control_group['body']]
print("Done")
df_anxiety_group['f-title-tokens'] = [get_tokens(title, True) for title in df_anxiety_group['title']]
print("Done")
df_anxiety_group['f-body-tokens'] = [get_tokens(body, True) for body in df_anxiety_group['body']]
print("Done")

Done
Done
Done
Done


SENTIMENT ANALYSIS:
https://textblob.readthedocs.io/en/dev/quickstart.html

In [199]:
def get_polarity(string): 
    '''  '''
    try:
        tb = TextBlob(string)
        return tb.sentiment.polarity # TODO : CHANGE THIS AS PER NEED
    except:
        return None

In [200]:
df_control_group['f-title-sentiment'] = [get_polarity(title) for title in df_control_group['title']]
print("Done")
df_control_group['f-body-sentiment'] = [get_polarity(body)  for body in df_control_group['body']]
print("Done")
df_anxiety_group['f-title-sentiment'] = [get_polarity(title)  for title in df_anxiety_group['title']]
print("Done")
df_anxiety_group['f-body-sentiment'] = [get_polarity(body)  for body in df_anxiety_group['body']]
print("Done")

Done
Done
Done
Done


In [207]:
df_control_group.head(10).to_html('temp.html')